In [1]:
import numpy as np
import time
import math
import pymysql as mysql

np.random.seed(int(time.time()))

In [2]:
def adjust_weights(weights, prev_draw, pick_most_freq=True):
    ratio = (1 + np.sqrt(5)) / 2 - 1
    new_weights = weights.copy()
    indices = np.array(prev_draw) - 1
    if not pick_most_freq:
        # Update the weight of numbers taht are NOT in previous draw.
        indices = np.delete(np.arange(len(weights)), indices)
    new_weights[indices] = new_weights[indices] + 1
    return new_weights

def load_history(lottery, latest_n=0):
    hist = []
    conn = mysql.connect(read_default_file="~/my.cnf")
    cursor = conn.cursor()
    sql = "SELECT draw_date, pick_5, pick_1 FROM lottery_draws WHERE lottery = %s ORDER BY draw_date DESC"
    if latest_n > 0:
        sql += " LIMIT " + str(latest_n)
    cursor.execute(sql, (lottery))
    for row in cursor.fetchall():
        hist.append({"pick_5": [int(n) for n in row[1].split(",")], "pick_1": [int(row[2])]})
    hist.reverse()
    return hist

def pick_lotteries(total_num, pick_5_max, pick_1_max, pick_most_freq=True, prev_draws=[]):
    pick_5_choices = np.array([i+1 for i in range(pick_5_max)])
    pick_5_weights = np.array([1.0 for _ in range(pick_5_max)])
    pick_1_choices = np.array([i+1 for i in range(pick_1_max)])
    pick_1_weights = np.array([1.0 for _ in range(pick_1_max)])
    picks = []
    
    for p in prev_draws:
        pick_5_weights = adjust_weights(pick_5_weights, p['pick_5'], pick_most_freq=pick_most_freq)
        pick_1_weights = adjust_weights(pick_1_weights, p['pick_1'], pick_most_freq=pick_most_freq)
    print("Pick 5 Var/Mean Ratio:", pick_5_weights.var() / pick_5_weights.mean())
    print("Pick 1 Var/Mean Ratio:", pick_1_weights.var() / pick_1_weights.mean())
    
    for i in range(total_num):
        prob = pick_5_weights / np.sum(pick_5_weights)
        pick_5 = np.random.choice(pick_5_choices, size=5, replace=False, p=prob)
        pick_5.sort()

        prob = pick_1_weights / np.sum(pick_1_weights)
        pick_1 = np.random.choice(pick_1_choices, size=1, p=prob)
        picks.append([pick_5.tolist(), pick_1.tolist()])
    return picks

In [12]:
TOTAL_NUM = 5
PICK_5_MAX = 70
PICK_1_MAX = 25
hist = load_history("MM", latest_n=4*PICK_5_MAX)
picks = pick_lotteries(TOTAL_NUM, PICK_5_MAX, PICK_1_MAX, pick_most_freq=True, prev_draws=hist)

print("\nMega Million lottery tickets")
for p in picks:
    print(" ".join(["{:2d}".format(n) for n in p[0]]), "|", "{:2d}".format(p[1][0]))

Pick 5 Var/Mean Ratio: 0.8598639455782313
Pick 1 Var/Mean Ratio: 0.48524590163934433

Mega Million lottery tickets
13 30 43 48 60 | 17
17 20 45 46 60 | 15
10 12 34 48 62 | 13
 7 12 35 62 70 | 16
14 49 56 66 67 | 13


In [13]:
TOTAL_NUM = 5
PICK_5_MAX = 69
PICK_1_MAX = 26
hist = load_history("PB", latest_n=8*PICK_5_MAX)
picks = pick_lotteries(TOTAL_NUM, PICK_5_MAX, PICK_1_MAX, pick_most_freq=True, prev_draws=hist)

print("\nPower Ball lottery tickets")
for p in picks:
    print(" ".join(["{:2d}".format(n) for n in p[0]]), "|", "{:2d}".format(p[1][0]))

Pick 5 Var/Mean Ratio: 0.8900671615411806
Pick 1 Var/Mean Ratio: 0.7000266169816344

Power Ball lottery tickets
 7 26 27 59 66 |  9
 1  2 23 44 61 | 24
18 30 47 51 59 | 10
28 37 48 49 64 | 24
 5 37 43 48 55 | 21


In [9]:
TOTAL_NUM = 5
PICK_5_MAX = 47
PICK_1_MAX = 27
hist = load_history("CA/SLP", latest_n=20*PICK_5_MAX)
picks = pick_lotteries(TOTAL_NUM, PICK_5_MAX, PICK_1_MAX, pick_most_freq=True, prev_draws=hist)

print("\nCalifornia Super Lotto Plus lottery tickets")
for p in picks:
    print(" ".join(["{:2d}".format(n) for n in p[0]]), "|", "{:2d}".format(p[1][0]))

Pick 5 Var/Mean Ratio: 0.7465767853381082
Pick 1 Var/Mean Ratio: 1.2410279980083496

California Super Lotto Plus lottery tickets
 9 11 13 16 47 | 12
 1  5  7 25 38 | 15
11 22 24 37 46 | 18
 9 17 22 31 40 | 25
 1  4  7 32 37 |  3
